In [1]:
import pandas as pd
import os
import pickle
import glob
from tqdm import tqdm
import numpy as np

# Exploit DB Metadat

In [2]:
exploitdb_df = pd.read_csv(os.path.join('data', 'exploitdb_metadata.csv'), index_col=0)

#we can disregard URL
exploitdb_df['CVE'] = exploitdb_df['CVE'].str.split('/').str[-1]


exploitdb_df[exploitdb_df.select_dtypes(include=bool).columns] *= 1

exploitdb_df.head()

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

,url,CVE,Verified,Type,HasApplication,Platform,Date,Critical
0,https://www.exploit-db.com/exploits/15501,CVE-2010-4837,1,webapps,0,PHP,2010-11-12,0
1,https://www.exploit-db.com/exploits/44028,CVE-2018-6889,0,webapps,0,PHP,2018-02-13,0
2,https://www.exploit-db.com/exploits/7046,CVE-2008-5654,1,webapps,0,PHP,2008-11-07,0
3,https://www.exploit-db.com/exploits/24904,CVE-2013-1493,1,remote,0,Windows,2013-03-29,0
4,https://www.exploit-db.com/shellcodes/43747,NaN,0,shellcode,0,Linux_x86,2009-01-01,0


In [3]:
exploitdb_df.columns

Index(['url', 'CVE', 'Verified', 'Type', 'HasApplication', 'Platform', 'Date',
       'Critical'],
      dtype='object')

In [4]:
#see averages in booleans
exploitdb_df.mean()

Verified          0.770134
HasApplication    0.179356
Critical          0.127373
dtype: float64

In [5]:
#percent missing
exploitdb_df.isna().mean()

url               0.000000
CVE               0.416230
Verified          0.000000
Type              0.002404
HasApplication    0.000000
Platform          0.002404
Date              0.002404
Critical          0.000000
dtype: float64

In [6]:
exploitdb_df['CVE'].nunique()

21230

In [7]:
exploitdb_df['Type'].value_counts()

webapps      24771
remote        7069
dos           6740
local         4379
shellcode     1026
Name: Type, dtype: int64

In [8]:
exploitdb_df['Platform'].nunique()

71

In [9]:
exploitdb_df['Platform'].value_counts()[:10]

PHP          20453
Windows       9785
Linux         2969
Multiple      2733
Hardware      1807
ASP           1539
CGI            775
Linux_x86      500
Unix           322
OSX            316
Name: Platform, dtype: int64

In [10]:
exploitdb_df['Platform'].value_counts()[:-10]

PHP         20453
Windows      9785
Linux        2969
Multiple     2733
Hardware     1807
            ...  
UnixWare        4
BSDi_x86        3
NodeJS          3
Lua             2
ULTRIX          2
Name: Platform, Length: 61, dtype: int64

# Exploit DB Critical CVEs

In [11]:
#get all that are verified and true and a CVE
exploitdb_critical_df = exploitdb_df[exploitdb_df['Critical']==True]

exploitdb_critical_df.head()

,url,CVE,Verified,Type,HasApplication,Platform,Date,Critical
13,https://www.exploit-db.com/exploits/5889,CVE-2008-2890,1,webapps,1,PHP,2008-06-21,1
23,https://www.exploit-db.com/exploits/2880,CVE-2009-0450,1,local,1,Windows,2006-12-01,1
26,https://www.exploit-db.com/exploits/40528,NaN,1,local,1,Windows,2016-10-13,1
33,https://www.exploit-db.com/exploits/5137,CVE-2008-0813,1,webapps,1,PHP,2008-02-17,1
37,https://www.exploit-db.com/exploits/32800,CVE-2009-0756,1,dos,1,Linux,2009-02-12,1


In [12]:
exploitdb_critical_df.isna().mean()

url               0.000000
CVE               0.425392
Verified          0.000000
Type              0.000000
HasApplication    0.000000
Platform          0.000000
Date              0.000000
Critical          0.000000
dtype: float64

In [13]:
#any over represented CVE?
exploitdb_critical_df['CVE'].value_counts()

CVE-2006-4985    22
CVE-2007-3217    12
CVE-2006-3019    10
CVE-2006-1205    10
CVE-2008-3664     9
                 ..
CVE-2009-5141     1
CVE-2015-7347     1
CVE-2008-0742     1
CVE-2006-5284     1
CVE-2007-1251     1
Name: CVE, Length: 2509, dtype: int64

In [14]:
exploitdb_critical_df['Type'].value_counts()

webapps    2811
remote     1023
dos         953
local       829
Name: Type, dtype: int64

In [15]:
exploitdb_critical_df['Platform'].value_counts()

PHP               2718
Windows           2027
Linux              470
Multiple           179
ASP                 65
Windows_x86         42
CGI                 24
Windows_x86-64      22
OSX                 17
Unix                17
BSD                  8
JSP                  5
Linux_x86            4
Java                 4
Solaris              3
AIX                  3
Python               3
Linux_x86-64         1
Ruby                 1
FreeBSD              1
ASPX                 1
Android              1
Name: Platform, dtype: int64

## 2 Sided KS Test

Hypothesis: Are the types following the same distributions?

In [16]:
from scipy import stats


In [17]:
merged_type_ks = pd.merge(exploitdb_df['Type'].value_counts().to_frame(), 
                     exploitdb_critical_df['Type'].value_counts().to_frame(),
                          suffixes= ('_all', '_critical'),
                          left_index=True, 
                          right_index=True)

stats.ks_2samp(merged_type_ks['Type_all'], merged_type_ks['Type_critical'])

Ks_2sampResult(statistic=1.0, pvalue=0.028571428571428577)

In [18]:
merged_type_ks['ratio_diff'] = (merged_type_ks['Type_critical'] / merged_type_ks['Type_all'])

merged_type_ks.sort_values(by='ratio_diff', ascending=False)

merged_type_ks.head(10)

,Type_all,Type_critical,ratio_diff
webapps,24771,2811,0.113479
remote,7069,1023,0.144716
dos,6740,953,0.141395
local,4379,829,0.189313


Hypothesis: Are the platforms following the same distributions?

In [19]:
merged_platform_ks = pd.merge(exploitdb_df['Platform'].value_counts().to_frame(), 
                     exploitdb_critical_df['Platform'].value_counts().to_frame(),
                          suffixes= ('_all', '_critical'),
                          left_index=True, 
                          right_index=True)

stats.ks_2samp(merged_platform_ks['Platform_all'], merged_platform_ks['Platform_critical'])

Ks_2sampResult(statistic=0.6818181818181818, pvalue=3.642468216632747e-05)

Index(['CVE', 'Date'], dtype='object')